In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pyautogui

---

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
actions = np.array(['idle_0','idle_1','idle_2','mouse_on','mouse_off', 'sleep','down_vol'])


In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
cap = cv2.VideoCapture(1)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        # Flip the camera
        frame = cv2.flip(frame, 1)

        print(frame.shape[1],frame.shape[0])
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [18]:
cap.release()
cv2.destroyAllWindows()

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [61]:
33*4+21*3+21*3

258

---

In [4]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['idle_0','idle_1','idle_2','mouse_on','mouse_off', 'sleep','using_mouse_L','using_mouse_R','using_mouse_LR'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [82]:
for action in ['idle_0','idle_1','idle_2','mouse_on','mouse_off', 'sleep','using_mouse_L','using_mouse_R','using_mouse_LR']:
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [10]:
cap.release()
cv2.destroyAllWindows()

In [83]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # NEW LOOP
    # Loop through actions
    for action in ['using_mouse_L','using_mouse_R','using_mouse_LR']:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

---

In [3]:
import tensorflow
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
label_map = {label:num for num, label in enumerate(actions)}

In [6]:
label_map

{'idle_0': 0,
 'idle_1': 1,
 'idle_2': 2,
 'mouse_on': 3,
 'mouse_off': 4,
 'sleep': 5,
 'using_mouse_L': 6,
 'using_mouse_R': 7,
 'using_mouse_LR': 8}

In [7]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [8]:
np.array(sequences).shape

(270, 30, 258)

In [9]:
X = np.array(sequences)

In [10]:
#One-hot encoding

y = to_categorical(labels).astype(int)

In [11]:
#Split train-testing with test size being 5% of total data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [93]:
X_test.shape

(14, 30, 258)

---

In [94]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [95]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [96]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(30,258)))
# model.add(LSTM(2048, return_sequences=True, activation='relu'))
# model.add(LSTM(1024, return_sequences=True, activation='relu'))
model.add(LSTM(512, return_sequences=True, activation='relu'))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [97]:
optim = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)

In [98]:
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [99]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
8/8 [==============================] - 6s 198ms/step - loss: 2.1740 - categorical_accuracy: 0.1836
Epoch 2/500
8/8 [==============================] - 1s 174ms/step - loss: 2.0646 - categorical_accuracy: 0.2266
Epoch 3/500
8/8 [==============================] - 2s 193ms/step - loss: 1.5872 - categorical_accuracy: 0.4023
Epoch 4/500
8/8 [==============================] - 1s 166ms/step - loss: 1.1827 - categorical_accuracy: 0.5703
Epoch 5/500
8/8 [==============================] - 1s 149ms/step - loss: 0.7866 - categorical_accuracy: 0.6875
Epoch 6/500
8/8 [==============================] - 1s 191ms/step - loss: 0.5642 - categorical_accuracy: 0.8438
Epoch 7/500
8/8 [==============================] - 1s 181ms/step - loss: 0.3690 - categorical_accuracy: 0.9023
Epoch 8/500
8/8 [==============================] - 2s 200ms/step - loss: 0.4755 - categorical_accuracy: 0.8828
Epoch 9/500
8/8 [==============================] - 2s 205ms/step - loss: 0.2985 - categorical_accuracy: 0.8945
E

KeyboardInterrupt: 

In [100]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_57 (LSTM)              (None, 30, 256)           527360    
                                                                 
 lstm_58 (LSTM)              (None, 30, 512)           1574912   
                                                                 
 lstm_59 (LSTM)              (None, 30, 256)           787456    
                                                                 
 lstm_60 (LSTM)              (None, 128)               197120    
                                                                 
 dense_25 (Dense)            (None, 64)                8256      
                                                                 
 dense_26 (Dense)            (None, 32)                2080      
                                                                 
 dense_27 (Dense)            (None, 9)               

---

In [101]:
model.save('action_v4.h5')

In [44]:
del model

In [13]:
from tensorflow.keras.models import load_model
model = load_model('Models/action_v4.h5')

---

In [14]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [15]:
yhat = model.predict(X_train)

8/8 [==============================] - 3s 60ms/step


In [16]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [17]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[227,   0],
        [  0,  29]],

       [[228,   0],
        [  0,  28]],

       [[227,   0],
        [  0,  29]],

       [[228,   0],
        [  0,  28]],

       [[226,   0],
        [  0,  30]],

       [[228,   0],
        [  0,  28]],

       [[227,   0],
        [  0,  29]],

       [[228,   0],
        [  0,  28]],

       [[229,   0],
        [  0,  27]]], dtype=int64)

In [18]:
accuracy_score(ytrue, yhat)

1.0